In [ ]:
# Python packages 

#!pip install import-ipynb
#!pip install git+https://github.com/patrick-kidger/torchcubicspline.git # this pip install shouldn't be strictly needed but kept just in case.

import import_ipynb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torchvision import datasets
from torchvision import transforms
import Mymodels

import Preprocess
from torchcubicspline import(natural_cubic_spline_coeffs, 
                             NaturalCubicSpline)
from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline
from scipy.optimize import newton,least_squares
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

In [ ]:
# Calls to Preprocess for loading of data

# Swap files prefix and postfix names
prefix = ['ad','cd','nk','sw','dk','EU','US','uk','jy']
postfix = ["1year.xlsx", "2year.xlsx", "3year.xlsx", "5year.xlsx", "10year.xlsx", "15year.xlsx", "20year.xlsx", "30year.xlsx"]

# Training data range
date_start = "2009-12-31"
date_end = "2023-01-01"

data_path = 'data/BloombergData/'

# Call to Preprocess module
x_train, x_val= Preprocess.data_clean(Preprocess.init_cleaning(prefix, postfix, data_path, date_start, date_end))

#Dividing the input data to get it in decimal
x_train = [x[1:]/100 for x in x_train]
x_val = [x[1:]/100 for x in x_val]

#Float64 conversion for higher precision (and because Torch complains if not)
x_val = torch.from_numpy(np.float64(x_val))
x_train = torch.from_numpy(np.float64(x_train))

# Tensorfy data and put into Train and Validation loader.
loader = torch.utils.data.DataLoader(dataset = x_train,
                                     batch_size = 32,
                                     shuffle = True
                                     )
val_loader = torch.utils.data.DataLoader(dataset = x_val,
                                     batch_size = 32,
                                     shuffle = True
                                     )

In [ ]:
# Calls to model and training of models from Mymodels module.

# Default maturities (should not be changed)
maturities = np.linspace(0,30,31,endpoint=True)
swap_maturities = [1,2,3,5,10,15, 20, 30]
maturities_torch = torch.from_numpy(np.array([1,2,3,5,10,15, 20, 30],dtype=np.float64))

# seed for deterministic starting weight in models.
seed = 6
Mymodels.set_seed(seed)

#define your model, and its parameters
in_features = 8
latent_dim = 2
aEL = Mymodels.FIRAutoEncoder(in_features,latent_dim)
criterion = Mymodels.CustomLoss()
optimizer = torch.optim.Adam(aEL.parameters(), lr=0.01)
scheduler = Mymodels.CustomLRScheduler(optimizer)
epochs = 5000 



# Call to training of the specified model, change function if you change model.
Vautoencoder, Vlosses, Voutputs, VvalLosses = Mymodels.train_ae(aEL, loader, epochs, optimizer, criterion, maturities,swap_maturities, scheduler, val_loader)

In [ ]:
# Model saving

#name you wish to save the model under
name = "Test_model" 
torch.save({"model_state_dict" : Vautoencoder.state_dict(), "optimizer_state_dict" :optimizer.state_dict()},  "models/model_{0}_{1}".format(epochs,name))  


In [ ]:
# Plot code for creating the training/validation progress of the model

xlist = np.linspace(0,5000, 100)
plt.plot(xlist, Vlosses[0::50],c="red", label="Training Loss")
plt.plot(xlist,VvalLosses[0::50],c="blue", label="Validation Loss")
plt.yscale("log")
plt.ylabel("Log Loss")
plt.xlabel("Epochs")
plt.legend()
plt.savefig("trainval_plot")